# Plant Pathology Inference Notebook

In [47]:
from fastai.vision.all import *
%config Completer.use_jedi = False
%matplotlib inline

seed = 42
set_seed(seed, reproducible=True)

In [56]:
path = Path('/media/xar/barracuda1/fast.ai/')
data_path = Path(path/'data'/'plant_pathology')
models_path = Path(path/'ds_comps'/'vision'/'plant_pathology'/'inference')

models = [models_path/f'plant_resnet{r}_vl_480px_wd0.{w}.pkl' for r in [18, 34, 50, 101, 152] for w in range(1,5)]
# models = [models_path/f'plant_resnet101_vl_480px_wd0.2.pkl']
models

[Path('/media/xar/barracuda1/fast.ai/ds_comps/vision/plant_pathology/inference/plant_resnet18_vl_480px_wd0.1.pkl'),
 Path('/media/xar/barracuda1/fast.ai/ds_comps/vision/plant_pathology/inference/plant_resnet18_vl_480px_wd0.2.pkl'),
 Path('/media/xar/barracuda1/fast.ai/ds_comps/vision/plant_pathology/inference/plant_resnet18_vl_480px_wd0.3.pkl'),
 Path('/media/xar/barracuda1/fast.ai/ds_comps/vision/plant_pathology/inference/plant_resnet18_vl_480px_wd0.4.pkl'),
 Path('/media/xar/barracuda1/fast.ai/ds_comps/vision/plant_pathology/inference/plant_resnet34_vl_480px_wd0.1.pkl'),
 Path('/media/xar/barracuda1/fast.ai/ds_comps/vision/plant_pathology/inference/plant_resnet34_vl_480px_wd0.2.pkl'),
 Path('/media/xar/barracuda1/fast.ai/ds_comps/vision/plant_pathology/inference/plant_resnet34_vl_480px_wd0.3.pkl'),
 Path('/media/xar/barracuda1/fast.ai/ds_comps/vision/plant_pathology/inference/plant_resnet34_vl_480px_wd0.4.pkl'),
 Path('/media/xar/barracuda1/fast.ai/ds_comps/vision/plant_pathology/inf

In [49]:
# test_pics = (data_path/'test_images').ls().sorted()
# im_path = test_pics[0]
# im = Image.open(im_path)
# im

Define `get_x` and `get_y` but this time have them pointing towards the test images.

In [50]:
def get_x(df): return data_path/'test_images'/df['image']
def get_y(df): return df['labels'].split()

## Generate Predictions

In [51]:
sample = pd.read_csv(data_path/'sample_submission.csv')
sample

,image,labels
0,85f8cb619c66b863.jpg,healthy
1,ad8770db05586b59.jpg,healthy
2,c7b03e718489f3ca.jpg,healthy


In [52]:
predictions = 0
for m in models:
    learner = load_learner(m, cpu=False).to_fp32()
    test_dl = learner.dls.test_dl(sample)
    preds, _ = learner.tta(dl=test_dl)
    predictions += preds
    
predictions /= len(models)
predictions

tensor([[ 1.5106, -3.0578, -6.7058, -6.7718, -3.3152, -2.6660],
        [-0.0440,  1.0811, -7.6678, -7.6717, -6.1289,  1.0281],
        [-3.7034,  3.8764, -6.0400, -8.5150, -7.8361, -2.5752]])

In [53]:
learner.dls.vocab

['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab']

In [54]:
vocab = learner.dls.vocab
preds_dec = [vocab[i] for i in predictions.argmax(1)]
preds_dec

['complex', 'frog_eye_leaf_spot', 'frog_eye_leaf_spot']

## Submission

In [55]:
sample['labels'] = preds_dec
sample.to_csv('submission.csv', index=False)
sample

,image,labels
0,85f8cb619c66b863.jpg,complex
1,ad8770db05586b59.jpg,frog_eye_leaf_spot
2,c7b03e718489f3ca.jpg,frog_eye_leaf_spot
